In [1]:
import pandas as pd
from sklearn import preprocessing
from datetime import timedelta
from math import sin, cos, sqrt, atan2, radians

# Rassemblement des données

In [2]:
trips = pd.read_csv("tgv/trips.txt", usecols=[0, 2]) # Liste de tous les arrêts fait pour une route donnée
routes = pd.read_csv("tgv/routes.txt", usecols=[0, 3]) # Liste des grands trajets (ou grandes routes)
trips_routes = trips.merge(routes, on="route_id").drop('route_id', axis=1) # On merge
stops = pd.read_csv("tgv/stops.txt", usecols=[0, 1, 3, 4])
stop_times = pd.read_csv("tgv/stop_times.txt", usecols=[0, 1, 2, 3]) # L'heure des arrêts (heure à laquelle tu arrives en gare et l'heure à laquelle tu pars)

data = trips_routes.merge(stop_times, on="trip_id")

# les trips ont des noms bizarres, du coup je leur donne un id de 0 à 4XXX
le = preprocessing.LabelEncoder() 
data["trip_id"] = le.fit_transform(data["trip_id"])

data = data.sort_values(["trip_id", "arrival_time"]).reset_index(drop=True)

# Je convertis toutes les heures en 'timedelta' pour pouvoir faire des opérations par la suite
data[["arrival_time", "departure_time"]] = data.loc[:, ["arrival_time", "departure_time"]].apply(lambda col: col.str.split(":"), axis=1).applymap(lambda col: timedelta(hours=int(col[0]), minutes=int(col[1]), seconds=int(col[2])))
data

,trip_id,route_long_name,arrival_time,departure_time,stop_id
0,0,Paris<>Vallées,0 days 08:08:00,0 days 08:08:00,StopPoint:OCETGV-87742007
1,0,Paris<>Vallées,0 days 08:26:00,0 days 08:26:00,StopPoint:OCETGV-87742361
2,0,Paris<>Vallées,0 days 08:40:00,0 days 08:40:00,StopPoint:OCETGV-87742320
3,0,Paris<>Vallées,0 days 08:51:00,0 days 08:51:00,StopPoint:OCETGV-87741306
4,0,Paris<>Vallées,0 days 12:42:00,0 days 12:42:00,StopPoint:OCETGV-87686006
...,...,...,...,...,...
21767,3989,Rhin-Rhône,0 days 11:50:00,0 days 11:53:00,StopPoint:OCETGV INOUI-87182014
21768,3989,Rhin-Rhône,0 days 12:21:00,0 days 12:29:00,StopPoint:OCETGV INOUI-87212027
21769,3989,Rhin-Rhône,0 days 13:24:00,0 days 13:29:00,StopPoint:OCETGV INOUI-87192039
21770,3989,Rhin-Rhône,0 days 13:47:00,0 days 13:51:00,StopPoint:OCETGV INOUI-87191007


In [3]:
def get_routes(trip: pd.DataFrame):
    """ Fonction qui va prendre une route avec tous les arrêts et la transforme en liste de trajets """
    df = pd.DataFrame({})
    t1, d_id = trip.iloc[0, [3, 4]]
    for idx, row in trip.reset_index(drop=True).drop(0).iterrows():
        t2, a_id = row.iloc[[2, 4]] 
        time = t2 - t1
        r = pd.Series([d_id, a_id, time.seconds])
        t1 = row.iloc[3]
        t1, d_id = t2, a_id
        df = df.append(r, ignore_index=True)
    df.columns = ["d_id", "a_id", "time"]
    return df

def compute_dist(geo_d, geo_a):
    """ Calcule la distance en fonctionde lat et long de deux points"""
    R = 6373.0
    lat1 = radians(geo_d[0])
    lon1 = radians(geo_d[1])
    lat2 = radians(geo_a[0])
    lon2 = radians(geo_a[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [4]:
new_data= pd.DataFrame({}, columns=["d_id", "a_id", "time"])
# Les trips c'est les routes (ex: Paris CDG - Marseille ST Charles), j'itère sur toutes les routes comme celle-ci
# Et je transforme tous les arrêts de ces trajets en nouvelle ligne pour mon dataframe final
for i in data["trip_id"].unique():
    new_data = new_data.append(get_routes(data.groupby("trip_id").get_group(i)), ignore_index=True)
    
# Je remplace les id des arrêts par leur nom
new_data = new_data.merge(stops, left_on='d_id', right_on="stop_id")
new_data = new_data.merge(stops, left_on='a_id', right_on="stop_id", suffixes=("_d", "_a")).drop(["a_id", "d_id", "stop_id_d", "stop_id_a"], axis=1)

# Je supprime les doublons (il y en a pas mal normalement)
new_data = new_data.drop_duplicates().reset_index(drop=True)
new_data = new_data.groupby(['stop_name_d','stop_name_a']).mean().reset_index()

In [5]:
# Calcul de distance et d'émission carbone
new_data["distance (km)"] = new_data.apply(lambda row: compute_dist((row["stop_lat_d"], row["stop_lon_d"]), (row["stop_lat_a"], row["stop_lon_a"])), axis=1)
new_data["gCO2e/personne"] = new_data["distance (km)"] * 2.4
new_data["prix"] = new_data["distance (km)"] * 0.11

In [6]:
# D'après ce site : https://ressources.data.sncf.com/explore/dataset/emission-co2-tgv/information/
# Sur un TGV l'émission de CO2 par personne est de 2,4 gCO2e * distance

In [7]:
# Tout beau !
new_data

,stop_name_d,stop_name_a,time,stop_lat_d,stop_lon_d,stop_lat_a,stop_lon_a,distance (km),gCO2e/personne,prix
0,Gare de Agde,Gare de Béziers,870.0,43.317574,3.466020,43.336225,3.219218,20.077270,48.185448,2.208500
1,Gare de Agde,Gare de Sète,930.0,43.317574,3.466020,43.412812,3.696396,21.430247,51.432592,2.357327
2,Gare de Agen,Gare de Bordeaux-St-Jean,4536.0,44.207972,0.620905,44.826540,-0.556194,115.969633,278.327120,12.756660
3,Gare de Agen,Gare de Montauban-Ville-Bourbon,2400.0,44.207972,0.620905,44.014639,1.341974,61.469807,147.527538,6.761679
4,Gare de Agen,Gare de Toulouse-Matabiau,3720.0,44.207972,0.620905,43.611464,1.453558,94.096219,225.830926,10.350584
...,...,...,...,...,...,...,...,...,...,...
715,Gare de Versailles-Chantiers,Gare de Rouen-Rive-Droite,6060.0,48.795566,2.135465,49.449030,1.094154,105.016575,252.039781,11.551823
716,Gare de Vitry-le-François,Gare de Bar-le-Duc,1640.0,48.717733,4.587232,48.773631,5.167016,42.976421,103.143411,4.727406
717,Gare de Vitry-le-François,Gare de Châlons-en-Champagne,1220.0,48.717733,4.587232,48.955509,4.348954,31.682972,76.039133,3.485127
718,Gare de Vitré,Gare de Laval,1230.0,48.122420,-1.211966,48.076252,-0.760939,33.895182,81.348436,3.728470


In [19]:
col = [["point_1", "point_1", "point_1", "point_2", "point_2", "point_2", "duree (min)", "distance (km)", 
        "empreinte carbone (gCO2)", "prix (euros)"], 
       ["nom", "latitude", "longitude", "nom", "latitude", "longitude", "", "", "", ""]]
col = pd.MultiIndex.from_arrays(col)
df_tgv = pd.DataFrame(columns=col)
df_tgv[[("point_1", "nom"), ("point_2", "nom")]] = new_data[["stop_name_d", "stop_name_a"]]
df_tgv[[("point_1", "latitude"), ("point_1", "longitude"), ("point_2", "latitude"), ("point_2", "longitude")]] = \
new_data[["stop_lat_d", "stop_lon_d", "stop_lat_a", "stop_lon_a"]].applymap(lambda x: round(x, 5))
df_tgv[("duree (min)", "")] = new_data["time"].apply(lambda x: round(x/60, 2))
df_tgv[("distance (km)", "")] = new_data["distance (km)"].apply(lambda x: round(x, 3))
df_tgv[("empreinte carbone (gCO2)", "")] = new_data["gCO2e/personne"].apply(lambda x: round(x, 1))
df_tgv[("prix (euros)", "")] = new_data["prix"].apply(lambda x: round(x, 2))
df_tgv

point_1                      \
                              nom  latitude longitude   
0                    Gare de Agde  43.31757   3.46602   
1                    Gare de Agde  43.31757   3.46602   
2                    Gare de Agen  44.20797   0.62090   
3                    Gare de Agen  44.20797   0.62090   
4                    Gare de Agen  44.20797   0.62090   
..                            ...       ...       ...   
715  Gare de Versailles-Chantiers  48.79557   2.13546   
716     Gare de Vitry-le-François  48.71773   4.58723   
717     Gare de Vitry-le-François  48.71773   4.58723   
718                 Gare de Vitré  48.12242  -1.21197   
719                 Gare de Vitré  48.12242  -1.21197   

                             point_2                     duree (min)  \
                                 nom  latitude longitude               
0                    Gare de Béziers  43.33623   3.21922       14.50   
1                       Gare de Sète  43.41281   3.69640       15.50   
2           Gare de Bordeaux-St-Jean  44.82654  -0.55619       75.60   
3    Gare de Montauban-Ville-Bourbon  44.01464   1.34197       40.00   
4          Gare de Toulouse-Matabiau  43.61146   1.45356       62.00   
..                               ...       ...       ...         ...   
715        Gare de Rouen-Rive-Droite  49.44903   1.09415      101.00   
716               Gare de Bar-le-Duc  48.77363   5.16702       27.33   
717     Gare de Châlons-en-Champagne  48.95551   4.34895       20.33   
718                    Gare de Laval  48.07625  -0.76094       20.50   
719                   Gare de Rennes  48.10352  -1.67233       22.75   

    distance (km) empreinte carbone (gCO2) prix (euros)  
                                                         
0          20.077                     48.2         2.21  
1          21.430                     51.4         2.36  
2         115.970                    278.3        12.76  
3          61.470                    147.5         6.76  
4          94.096                    225.8        10.35  
..            ...                      ...          ...  
715       105.017                    252.0        11.55  
716        42.976                    103.1         4.73  
717        31.683                     76.0         3.49  
718        33.895                     81.3         3.73  
719        34.253                     82.2         3.77  

[720 rows x 10 columns]

In [20]:
df_tgv.to_csv("results/df_tgv.csv", index=False)